In [38]:
%matplotlib inline

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

This process will bring together the data sources so that they will be more easily queryable for our investigation later. the intention is to create two files. Each row in the these files is California school. each school will have the demographic information outlined in the proposal. the first file will contain the data for schools that for which migrant data is available. The Second file will contain this demographic information for every California school.

****

## Transformations

These dataframes will be created in parallel because the processes are nearly identicle. First remove any nulls, in this case some of the scores are starred out.



**1. Mean Score by School**
+ Mean Scale score is agregated using the groupby function to give an average score for each school 

**2. Entities**
+ the dataframe from the previous step is joined with information about the schools location that will be used to determine sanctuary status.

**3. Per Pupil Spending**
+ Data about the schools per pupil spending is joined via the district code

**4. Local Median Income**
+ Data about the median income in the schools zip code 

**5. Education Experience**
+ records of employees are agregated at the district level to give a metric for the average number of years experience a teacher has.

**6. Food Aid Utilization**
+ at the school level information about both the availability and actual utilization of free and reduced priced meals aimed at those struggling with food insecurity

**7. Staff Ethnicity**
+ for each school district the ratios of the 10 available ethnicities is calculated, on the district code 10 more columns are added to represent the frequency of the ethnic groups.



****
**1. Mean Score by School **

clean for non numeric inputs, then the goal is to create a list of all school code with the average mean scale score.

****

In [39]:
#create a data frame every test for every grade for every school
Test_Results = pd.read_csv('Data/ca2016_all.csv')
Test_Results = Test_Results[Test_Results['Mean Scale Score']!='*']
Test_Results['Mean Scale Score'] = pd.to_numeric(Test_Results['Mean Scale Score'])
Test_Results = Test_Results[Test_Results['Mean Scale Score']>100]
len(Test_Results)

58009

In [40]:
Migrant_Test_Results = Test_Results[Test_Results['Subgroup ID']==28]
Migrant_School_List = Migrant_Test_Results.groupby('School Code')['Mean Scale Score'].mean().reset_index()
len(Migrant_School_List)
#here is basis on which the rest of the DataFrame will be built

379

In [41]:
#creating that same dataframe but for all schools
All_Test_Results = Test_Results[Test_Results['Subgroup ID']==1]
All_School_List = All_Test_Results.groupby('School Code')['Mean Scale Score'].mean().reset_index()
All_School_List.head(3)

,School Code,Mean Scale Score
0,100016,2497.525
1,100024,2520.000
2,100040,2537.900


As expected we have many more schools filled in for our all school list than our migrant school list

In [42]:
print("Migrant size: ",len(Migrant_School_List)," All size: ", len(All_School_List))

Migrant size:  379  All size:  9207


****
**2. Entities **

Join both list with thier location and district information using school code in the Entities file

****

In [43]:
Entities = pd.read_csv('Data/sb_ca2016entities.csv')
Migrant_School_List = Migrant_School_List.merge(Entities[['County Name','School Code','District Code','Zip Code','District Name']],on='School Code')
Migrant_School_List = Migrant_School_List[['Mean Scale Score','County Name','School Code','District Code','Zip Code','District Name']]
print(len(Migrant_School_List))
Migrant_School_List.head(2)

379


,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name
0,2520.025,Sacramento,100040,67348,95632,Galt Joint Union Elementary School Distr
1,2515.700,Kern,100198,63529,93307,Kern High School District


In [44]:
#now repeat this process for the dataframe with all schools
All_School_List = All_School_List.merge(Entities[['County Name','School Code','District Code','Zip Code','District Name']],on='School Code')
All_School_List = All_School_List[['Mean Scale Score','County Name','School Code','District Code','Zip Code','District Name']]
All_School_List.head(2)

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name
0,2497.525,Madera,100016,65243,93638,Sherman Thomas Charter
1,2520.000,El Dorado,100024,61838,95762,Buckeye Union Elementary School District


****
**3. Per Pupil Spending **

Import the per pupil spending by district and merge it the existing DataFrame. this will also generate a column about the number of students who regularly attend that school

****

In [45]:
Per_Pupil_Spending = pd.read_csv('Data/currentexpense1617.csv')
Per_Pupil_Spending.head()
#merge each DataFrame on District code to give each school is district funding information

,District Code,Total_Students,Per_Student_Spending
0,61119,9061.77,11570.19
1,61127,3601.47,12165.25
2,61143,9171.43,15503.92
3,61150,9022.97,10119.93
4,61168,644.25,17105.60


In [46]:
Per_Pupil_Spending.columns=['District Code','Total_Students','Per_Student_Spending']
Migrant_School_List = Migrant_School_List.merge(Per_Pupil_Spending[['District Code','Total_Students','Per_Student_Spending']])
print(len(Migrant_School_List))
Migrant_School_List.head(2)


372


,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Students,Per_Student_Spending
0,2520.0250,Sacramento,100040,67348,95632,Galt Joint Union Elementary School Distr,3455.23,11441.71
1,2419.7625,Sacramento,6033310,67348,95632,Galt Joint Union Elementary School Distr,3455.23,11441.71


In [47]:

All_School_List = All_School_List.merge(Per_Pupil_Spending)
All_School_List.head(2)

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Students,Per_Student_Spending
0,2497.525,Madera,100016,65243,93638,Sherman Thomas Charter,19011.95,10950.81
1,2465.150,Madera,107938,65243,93638,Ezequiel Tafoya Alvarado Academy,19011.95,10950.81


****
**4. Local Median Income **

Similiar to the previous step we will load in extra data from a file and merge. The key difference here is that Median Income is merged on the Zip Code

****

In [48]:
Zipcode_Income = pd.read_csv('Data/zipcode_income.csv')
Zipcode_Income['Zip Code'] = Zipcode_Income['Zip Code'].astype(str)
Zipcode_Income.head(1)

,Zip Code,Median,Mean,Pop
0,1001,56663.0,66688.00,16445.0


In [49]:
Migrant_School_List  = Migrant_School_List.merge(Zipcode_Income)
All_School_List  =All_School_List.merge(Zipcode_Income)
#check for data loss
print(len(Migrant_School_List))
All_School_List.head(1)
#now both DataFrames have the Apropriate Income Data

368


,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Students,Per_Student_Spending,Median,Mean,Pop
0,2497.525,Madera,100016,65243,93638,Sherman Thomas Charter,19011.95,10950.81,40096.0,52034.00,49068.0


****
**5. Educational Expereince **

The next file is a list of all current employees, what district they work in and how many years they have been a teacher. the average years will be created by grouping by District Code.

****

In [50]:
Staff_List = pd.read_csv('Data/Staff_Files.csv')
Staff_List['District Code'] =  Staff_List['District Code'].astype(int)
Staff_List.head(1)

,AcademicYear,RecID,District Code,County Name,District Name,GenderCode,Age,EducationLevel,EthnicGroup,YearsTeaching,YearsInDistrict,EmploymentStatusCode,FTE Teaching,FTE administrative,FTE PupilServices,FileCreated
0,1617,1118544,1964329,LOS ANGELES,Bonita Unified,M,53.0,V,7.0,21.0,13.0,T,45.0,0.0,0.0,1/16/2018


In [51]:


#Migrant_School_List  = Migrant_School_List.merge(District_Experience)
#check for data loss
#print(len(Migrant_School_List))
#print(len(District_Experience))

In [52]:
District_Experience = Staff_List.groupby('District Name')['YearsTeaching'].mean().reset_index()
Migrant_School_List['District Name'] = Migrant_School_List['District Name'].str.upper()
Migrant_School_List['District Name'] = Migrant_School_List['District Name'].str.slice(0,10)
District_Experience['District Name'] = District_Experience['District Name'].str.upper()
District_Experience['District Name'] = District_Experience['District Name'].str.slice(0,10)

All_School_List['District Name'] = All_School_List['District Name'].str.upper()
All_School_List['District Name'] = All_School_List['District Name'].str.slice(0,10)

Migrant_School_List = Migrant_School_List.merge(District_Experience,on = 'District Name')
Migrant_School_List = Migrant_School_List.drop_duplicates('School Code')
All_School_List  = All_School_List.merge(District_Experience,on = 'District Name')
Migrant_School_List = Migrant_School_List.drop_duplicates('School Code')

#check for data loss
print(len(Migrant_School_List))
#District_Experience.head(10)

361


****
**6. Food Aid Utilization **

This data will be joined at the school level. This is a measure of how much food assistance is being used by the various schools

****

In [53]:
Food_Aid = pd.read_csv('Data/Food_Assistance.csv')
Food_Aid.columns = ['School Code','Enrolled','Utilized','Percent_Eligible']
Food_Aid = Food_Aid.rename(index=str, columns={"Enrolled (K-12)": "Enrolled", "Utilized (K-12)": "Utilized","Percent_Eligable Eligible FPRM (K-12)":"Percent_Eligible"})
Food_Aid['Percent_Utilized'] = Food_Aid['Utilized']/ Food_Aid['Enrolled']
Food_Aid.head(1)

,School Code,Enrolled,Utilized,Percent_Eligible,Percent_Utilized
0,112607,413,308,0.75,0.745763


In [54]:
Migrant_School_List = Migrant_School_List.merge(Food_Aid)
All_School_List = All_School_List.merge(Food_Aid)
#check for data loss
print(len(Migrant_School_List))
All_School_List.head(2)

361


,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Students,Per_Student_Spending,Median,Mean,Pop,YearsTeaching,Enrolled,Utilized,Percent_Eligible,Percent_Utilized
0,2491.275,Madera,109694,65243,93638,MADERA UNI,19011.95,10950.81,40096.0,52034.00,49068.0,13.164474,897,814,0.91,0.907469
1,2425.300,Madera,109702,65243,93638,MADERA UNI,19011.95,10950.81,40096.0,52034.00,49068.0,13.164474,767,726,0.95,0.946545


****
**8. Sanctuary Status **

Using a list of sanctuary county search through and label each school in a sanctuary county as a 1

****

In [55]:
sanctuaries  = {'Alameda','Contra Costa','Los Angeles','Monterey','Napa','Riverside','Sacramento','San Bernardino','San Diego','San Francisco','San Mateo','Santa Clara','Santa Cruz','Sonoma'
}


In [56]:
#now we update this new sanctuary column based on whether the county is in our sanctuary list
All_School_List['Sanctuary'] = np.empty(len(All_School_List))
for i in range(len(All_School_List)):
  
    if All_School_List.ix[i,'County Name'] in sanctuaries:
        All_School_List.ix[i,'Sanctuary'] = 1
    else:
        All_School_List.ix[i,'Sanctuary'] = 0
    #print(school_list.ix[i,'sanctuary'])
All_School_List.head(1)

c:\users\admin\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
c:\users\admin\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
c:\users\admin\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Students,Per_Student_Spending,Median,Mean,Pop,YearsTeaching,Enrolled,Utilized,Percent_Eligible,Percent_Utilized,Sanctuary
0,2491.275,Madera,109694,65243,93638,MADERA UNI,19011.95,10950.81,40096.0,52034.00,49068.0,13.164474,897,814,0.91,0.907469,0.0


In [57]:
#now we update this new sanctuary column based on whether the county is in our sanctuary list
Migrant_School_List['Sanctuary'] = np.empty(len(Migrant_School_List))
for i in range(len(Migrant_School_List)):
  
    if Migrant_School_List.ix[i,'County Name'] in sanctuaries:
        Migrant_School_List.ix[i,'Sanctuary'] = 1
    else:
        Migrant_School_List.ix[i,'Sanctuary'] = 0
    #print(school_list.ix[i,'sanctuary'])
Migrant_School_List.head(1)

c:\users\admin\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
c:\users\admin\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
c:\users\admin\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Students,Per_Student_Spending,Median,Mean,Pop,YearsTeaching,Enrolled,Utilized,Percent_Eligible,Percent_Utilized,Sanctuary
0,2520.025,Sacramento,100040,67348,95632,GALT JOINT,3455.23,11441.71,64144.0,73007.00,29269.0,13.68,929,448,0.48,0.482239,1.0


In [58]:
#so How many of our 300 Migrant Schools fall in a sanctuary County
print(Migrant_School_List['Sanctuary'].count())

361


****
**9. Staff Ethnicity **

This data will be joined at the school level. This is a measure of how much food assistance is being used by the various schools

****

Staff_totals = Staff_List.groupby('District Name')['Age']
Staff_totals.head()

****
**10. Staff Age **

This data will be joined at the school level. This is a measure of the average age of the teaching staff

****


In [59]:
District_Experience = Staff_List.groupby('District Name')['Age'].mean().reset_index()
Migrant_School_List['District Name'] = Migrant_School_List['District Name'].str.upper()
Migrant_School_List['District Name'] = Migrant_School_List['District Name'].str.slice(0,10)
District_Experience['District Name'] = District_Experience['District Name'].str.upper()
District_Experience['District Name'] = District_Experience['District Name'].str.slice(0,10)

All_School_List['District Name'] = All_School_List['District Name'].str.upper()
All_School_List['District Name'] = All_School_List['District Name'].str.slice(0,10)

Migrant_School_List = Migrant_School_List.merge(District_Experience,on = 'District Name')
Migrant_School_List = Migrant_School_List.drop_duplicates('School Code')
All_School_List  = All_School_List.merge(District_Experience,on = 'District Name')
Migrant_School_List = Migrant_School_List.drop_duplicates('School Code')

#check for data loss
print(len(Migrant_School_List))
#District_Experience.head(10)

361


****
**11. Area Crime **

This data will be joined at the school level. This is a measure of how much food assistance is being used by the various schools

****

** 12. Score Percentiles **

In [60]:
#convert mean scale score to a target column that that will take a value 1-10 to represent which percentile the school falls in.

#All_School_List['Mean Scale Score'] = np.percentile(All_School_List['Mean Scale Score'])
All_School_List['Percentile'] = [stats.percentileofscore(All_School_List['Mean Scale Score'], a, 'rank') for a in All_School_List['Mean Scale Score']]
All_School_List['Percentile'] =  All_School_List['Percentile'] / 10
All_School_List['Percentile'] =   All_School_List['Percentile'].astype(int)
All_School_List.head(4)

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Students,Per_Student_Spending,Median,Mean,Pop,YearsTeaching,Enrolled,Utilized,Percent_Eligible,Percent_Utilized,Sanctuary,Age,Percentile
0,2491.2750,Madera,109694,65243,93638,MADERA UNI,19011.95,10950.81,40096.0,52034.00,49068.0,13.164474,897,814,0.91,0.907469,0.0,43.717105,5
1,2425.3000,Madera,109702,65243,93638,MADERA UNI,19011.95,10950.81,40096.0,52034.00,49068.0,13.164474,767,726,0.95,0.946545,0.0,43.717105,1
2,2422.9125,Madera,110957,65243,93638,MADERA UNI,19011.95,10950.81,40096.0,52034.00,49068.0,13.164474,745,678,0.91,0.910067,0.0,43.717105,1
3,2444.8375,Madera,113050,65243,93638,MADERA UNI,19011.95,10950.81,40096.0,52034.00,49068.0,13.164474,730,663,0.91,0.908219,0.0,43.717105,2


In [61]:
#Do the same for the Migrant Dataframe

#print(len(All_School_List['Mean Scale Score']))
Migrant_School_List['Percentile'] = [stats.percentileofscore(All_School_List['Mean Scale Score'].append(pd.Series([b])), b, 'rank') for b in Migrant_School_List['Mean Scale Score']]
Migrant_School_List['Percentile'] =  Migrant_School_List['Percentile'] / 10
Migrant_School_List['Percentile'] = Migrant_School_List['Percentile'].astype(int)

#print(len(All_School_List['Mean Scale Score']))
Migrant_School_List.head(3)


,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Students,Per_Student_Spending,Median,Mean,Pop,YearsTeaching,Enrolled,Utilized,Percent_Eligible,Percent_Utilized,Sanctuary,Age,Percentile
0,2520.0250,Sacramento,100040,67348,95632,GALT JOINT,3455.23,11441.71,64144.0,73007.00,29269.0,13.68,929,448,0.48,0.482239,1.0,43.64,7
2,2419.7625,Sacramento,6033310,67348,95632,GALT JOINT,3455.23,11441.71,64144.0,73007.00,29269.0,13.68,567,463,0.82,0.816578,1.0,43.64,1
4,2535.2500,Sacramento,3433471,67355,95632,GALT JOINT,2049.42,11850.39,64144.0,73007.00,29269.0,13.68,947,645,0.68,0.681098,1.0,43.64,7


In [62]:
# to finish, export the two new dataframes

In [63]:
Migrant_School_List.head()

,Mean Scale Score,County Name,School Code,District Code,Zip Code,District Name,Total_Students,Per_Student_Spending,Median,Mean,Pop,YearsTeaching,Enrolled,Utilized,Percent_Eligible,Percent_Utilized,Sanctuary,Age,Percentile
0,2520.0250,Sacramento,100040,67348,95632,GALT JOINT,3455.23,11441.71,64144.0,73007.00,29269.0,13.680000,929,448,0.48,0.482239,1.0,43.640000,7
2,2419.7625,Sacramento,6033310,67348,95632,GALT JOINT,3455.23,11441.71,64144.0,73007.00,29269.0,13.680000,567,463,0.82,0.816578,1.0,43.640000,1
4,2535.2500,Sacramento,3433471,67355,95632,GALT JOINT,2049.42,11850.39,64144.0,73007.00,29269.0,13.680000,947,645,0.68,0.681098,1.0,43.640000,7
6,2430.0750,Kern,100685,63503,93307,GREENFIELD,8995.08,10319.86,35780.0,43139.00,82658.0,15.344262,1014,899,0.89,0.886588,0.0,43.967213,1
8,2490.6000,Kern,101014,63503,93307,GREENFIELD,8995.08,10319.86,35780.0,43139.00,82658.0,15.344262,1048,929,0.89,0.886450,0.0,43.967213,5


In [64]:
len(Migrant_School_List)

361

In [65]:
Migrant_School_List.to_csv('Data/Migrant_School_List.csv')
All_School_List.to_csv('Data/All_School_List.csv')